In [25]:
!pip install antigranular &> /dev/null


import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

tls_cert_name: ip-100-100-15-185.eu-west-1.compute.internal_11589e76-6316-4116-a289-97064b78daf5
cert_thumbprint: d90875bf23925e6fcad143593f877711241236f409afc7d935fb60ce8afd65154f67bca43e0d829fb70948dfdb74a4aee265362e9f5a436ef478c6ae8fc29348
server_hostname: ip-100-100-15-185.eu-west-1.compute.internal
local_host_port: 11589e76-6316-4116-a289-97064b78daf5


IOStream.flush timed out


Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: 5e17ee57-103c-41e8-98e9-7d1cc498e9de, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [26]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [27]:
%%ag
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from op_pandas import standard_scaler, PrivateDataFrame, train_test_split

model = Sequential([
    # Giriş katmanı: 9 özellik
    Dense(512, activation='relu', input_shape=(9,)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

# Create DP keras model
dp_model = PrivateKerasModel(model=model, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #0.0003 #0.0003) 0.01 1

# PrivateKerasModel uses similar API as standard Keras, we compile
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2025-02-27 12:41:12.972773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2025-02-27 12:41:12.975863: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 12:41:13.161973: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 12:41:13.162014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [28]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=512)#128 256

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=50, target_delta=0.0001) #0.03 0.001

In [30]:
%%ag
x_test=PrivateDataFrame(x_test)

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)